## Vorbereitung LM Studio

Zunächst müssen wir LM Studio installieren und mindestens ein LLM herunterladen. Als Beispiel hier habe ich das LLAMA 3.1 Model (8B) geladen.

Dieses Modell müssen wir in LM Studio lokal starten. Es läuft dann lokal auf dem Rechner, und wird automatisch gefunden.

Dafür (und sowieso) brauchen wir die richtige Library und müssen das Modell instantiieren.

In [1]:

import lmstudio as lms

model = lms.llm("meta-llama-3.1-8b-instruct")

### Wir probieren aus ob das prinzipiell klappt 

In [2]:
result = model.respond("Who are you?")
print(result)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


## Laden der Reden

Wir wollen ja, das LLM benutzen, um die Reden zu charakterisieren. Also müssen wir die Reden laden. Das machen wir wie immer.

In [3]:
import jsonlines

legislatur = 20

alleReden = []
with jsonlines.open(f'../../data/speeches_{legislatur}.jsonl') as f:
    for line in f.iter():
        #for line in list(f):
        alleReden.append(line)

alleReden.sort(key = lambda x:x['date'])
print(len(alleReden))

25643


In [4]:
welche = 102
eineRede = alleReden[welche]
print(eineRede['text'])

Von mir aus nicht, Herr Spahn. Bleiben Sie ruhig sitzen. Herr Spahn, ich finde das eigentlich gut, was Sie gesagt haben, weil wir jetzt in ein Gespräch kommen, und das biete ich der Union, das bieten wir alle der Union an, auch über diese Abstimmung heute hinaus. Wir sind schon im Hauptausschuss auf die Union zugegangen. Ich habe es in vier Jahren Opposition nicht einmal erlebt – nicht einmal! –, dass wir noch während einer laufenden Sitzung eine Änderung an einem Vorschlag der Regierungsfraktionen vorgenommen haben. Das ist diese Woche passiert. Ich weiß nicht, wie weit wir noch auf Sie zugehen sollen. Aber woran es krankt, ist: Sie machen keine konkreten Vorschläge. Sie sagen uns nicht, was genau Sie wollen. Kommen Sie auf uns zu, und wir reden gerne weiter. Vielen Dank. – Als nächster Redner hat jetzt das Wort für die FDP-Fraktion Michael Theurer. 


## Eine erste Anfrage

In [5]:
system_msg = 'Du bist eine Experte in deutscher Politik. Dein Ziel ist es, den Menschen in Deutschland aktuelle politische Diskussionen verständlicher zu machen.'

user_msg = f"""Kannst mir den Inhalt folgender Bundestagsrede zusammenfassen? Hier ist die Rede: '{eineRede['text']}' """

chat = lms.Chat()
chat.add_system_prompt(system_msg)
chat.add_user_message(user_msg)
result = model.respond(chat)

print(result)

Ich werde dir den Inhalt der Rede zusammenfassen.

Die Rede ist von einer Fraktionskollegin in der Bundesversammlung abgehalten worden, der weder der CDU noch der FDP angehört hat. Der Sprecher spricht in dieser Rede über das Gespräch mit Herrn Spahn. 

Sie finden es gut, dass er gesprochen hat und biete die Opposition darin an, weiterhin zu sprechen, auch über diese Abstimmung hinaus. Sie haben ja schon im Hauptausschuss auf die Union zugegangen und sind bereit, noch mehr auf sie zuzugehen.

Der Grund dafür ist aber, dass Herr Spahn keine konkreten Vorschläge gemacht hat. Er sagt ihnen nicht genau, was er will. Sie laden ihn ein, auf sie zu kommen, um weiter zu sprechen. 

Sie bedanken sich im Anschluss bei Herrn Theurer für die Übergabe des Wortes und beginnen mit ihren eigenen Gedanken an der FDP.


## Let's play!

In [8]:
welche = 20201
eineRede = alleReden[welche]
print(eineRede['text'])

Sehr geehrte Frau Präsidentin! Liebe Kolleginnen und Kollegen! In kaum einem anderen Bereich klaffen Anspruch und Wirklichkeit so weit auseinander wie in der Menschenrechtspolitik der Bundesregierung. Im Iran werden täglich grundlegende Menschenrechte verletzt, doch die Bundesregierung reagiert darauf nur halbherzig. Unsere Außenministerin bietet Showpolitik mit starken Sätzen, denen aber keine Taten folgen. Wir warten immer noch darauf, dass die iranischen Revolutionsgarden als Terrororganisation gelistet werden. In Gaza wurden in sechs Monaten Krieg Tausende von Zivilistinnen getötet, über 1 Million Frauen und Mädchen wurden laut UN Women vertrieben – von feministischer Außenpolitik keine Spur. Die Bundesregierung muss endlich ihrer völkerrechtlichen Verantwortung gerecht werden. Sie muss sich klar und deutlich für einen Waffenstillstand in Israel und Gaza einsetzen. Der Export von Rüstungsgütern nach Israel und in alle anderen Kriegs- und Krisenregionen muss gestoppt werden. Auch im

In [10]:

# Liste möglicher Themen im Bundestag
THEMEN_KATEGORIEN = [
    "Außenpolitik und internationale Beziehungen",
    "Verteidigung und Bundeswehr",
    "Europapolitik",
    "Wirtschaft und Industrie",
    "Arbeitsmarkt und Beschäftigung",
    "Finanzen und Steuern",
    "Gesundheit und Pflege",
    "Bildung und Forschung",
    "Digitalisierung und Technologie",
    "Klimaschutz und Umwelt",
    "Energiepolitik",
    "Verkehr und Infrastruktur",
    "Soziale Gerechtigkeit und Wohlfahrt",
    "Familie und Gleichstellung",
    "Migration und Integration",
    "Innere Sicherheit und Polizei",
    "Justiz und Rechtsstaat",
    "Demokratie und Parlamentarismus",
    "Verbraucherschutz",
    "Landwirtschaft und Ernährung",
    "Wohnungsbau und Mieten",
    "Kultur und Medien",
    "Religionspolitik",
    "Entwicklungspolitik",
    "Sport",
    "Bildungspolitik (Schule, Hochschule)",
    "Rentensystem und Altersvorsorge",
    "Steuerpolitik",
    "Bundeshaushalt",
    "Ländliche Räume und Strukturpolitik"
]

kategorien = "\n".join(f"- {k}" for k in THEMEN_KATEGORIEN)

system_msg = 'Du bist ein politischer Analyst und klassifizierst Bundestagsreden nach Thema und Haltung.'

user_msg = f"""
Analysiere die folgende Rede aus dem Deutschen Bundestag.

1. Wähle **ein Thema** aus der folgenden Liste, das am besten zur Rede passt:
{kategorien}

2. Klassifiziere die **Haltung (Stance)** des Redners zu diesem Thema:
   – Zustimmend / Unterstützend
   – Ablehnend / Kritisch
   – Neutral / Ausgewogen

3. Gib eine kurze Begründung auf Basis des Redeinhalts (Zitat oder sinngemäße Aussage).

Gib die Antwort im JSON-Format zurück:

{{
  "Thema": "...",
  "Stance": "...",
  "Begründung": "..."
}}

### Rede:
{eineRede['text']}
""".strip()

chat = lms.Chat()
chat.add_system_prompt(system_msg)
chat.add_user_message(user_msg)
result = model.respond(chat)

print(result)


{
  "Thema": "Menschenrechtspolitik und internationale Beziehungen",
  "Stance": "Ablehnend / Kritisch",
  "Begründung": "Der Redner kritisiert die Bundesregierung für ihre 'halbherzige' Reaktion auf Menschenrechtsverletzungen in Iran und Gaza, fordert einen Waffenstillstand und den Export von Rüstungsgütern zu stoppen und hebt auch die mangelhafte Umsetzung der Istanbul-Konvention und die Abschaffung des § 218 Strafgesetzbuch als notwendig für Menschenrechte auf. Der Redner wirkt kritisch gegenüber der aktuellen Politik der Bundesregierung."
}
